In [7]:
import sys; sys.path.append('../..')
from abslithist import *

In [19]:
C = lltk.load('CanonFiction')
# C.meta

In [9]:

NLP_PROPS={
    'quote.singleQuotes': True,
    'quote.asciiQuotes': True,
    'quote.attributeQuotes': True
}
NLP_ANNOS = "tokenize,ssplit,pos,lemma,ner,parse,depparse,coref,quote"

def get_nlp_client(properties=NLP_PROPS, annotators=NLP_ANNOS, output_format='json', verbose=False, threads=None):
    global NLP_CLIENT
    if NLP_CLIENT is None:
        from stanza.server import CoreNLPClient
        import multiprocessing as mp
        if threads is None: threads=mp.cpu_count()
        NLP_CLIENT = CoreNLPClient(properties=properties, annotators=annotators, output_format=output_format, be_quiet=not verbose, threads=threads)
        NLP_CLIENT.start()
    return NLP_CLIENT

def stop_nlp_client():
    global NLP_CLIENT
    if NLP_CLIENT is not None:
        NLP_CLIENT.stop()
        NLP_CLIENT = None

In [10]:
stop_nlp_client()
nlp = get_nlp_client()
# odx=nlp.annotate(txt)

Writing properties to tmp file: corenlp_server-0a9c5eb062d94726.props
Starting server with command: java -Xmx5G -cp /Users/ryan/stanza_corenlp/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 60000 -threads 10 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-0a9c5eb062d94726.props -annotators tokenize,ssplit,pos,lemma,ner,parse,depparse,coref,quote -preload -outputFormat json


In [19]:
def save_text_nlp(text, force=False, resume=True, lim_paras=10, shuffle_paras=False, progress=True, nlp_max_chars=3000):
    # paths
    opath = text.path_nlp
    odir = os.path.dirname(opath)
    if not os.path.exists(odir):
        os.makedirs(odir)
    
    # start off where we were?
    done_paras_i = set()
    if os.path.exists(opath):
        if not force: return
        if resume:
            with open(opath) as f:
                for ln in f:
                    para_i=ln.strip().split('\t',1)[0]
                    if para_i and para_i.isdigit():
                        done_paras_i|={int(para_i)}
    
    # get paragraphs
    paras = [(i+1, para) for i,para in enumerate(text.paras)]
    if not force: paras = [(para_i,para) for para_i,para in paras if not para_i in done_paras_i]
    if shuffle_paras: random.shuffle(paras)
    if lim_paras: paras = paras[:lim_paras]    
    
    # pipe through nlp
    nlp = get_nlp_client()
    
    mode = 'a+b' if resume else 'wb'
    with open(opath,mode) as of:
        for para_i,para in tqdm(paras,disable=not progress):
            try:
                odx = nlp.annotate(para[:nlp_max_chars])
            except Exception as e:
                # print('!!',e)
                continue
            ob = orjson.dumps(odx)
            oln = f'{para_i}\t'.encode() + ob + b'\n'
            of.write(oln)

In [13]:
for x in pmap_iter(save_text_nlp, C.texts(), num_proc=1): pass

Mapping save_text_nlp() [x1]:   1%|          | 11/1805 [3:45:45<613:39:23, 1231.42s/it]


KeyboardInterrupt: 

In [55]:
stop_nlp_client()